In [1]:
# Various pip
!pip install tensorflow
!pip install transformers
!pip install datasets
!pip install wordfreq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.4 MB/s eta 0:00:00


In [78]:
# Dataframe management
import pandas as pd

# Data manipulation
import numpy as np
import random

# NNs
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Transformers
import transformers

# Convert to HuggingFace dataset
import pyarrow as pa
from datasets import Dataset
import datasets

# Sklearn for train test split
from sklearn.model_selection import train_test_split

# Wordfreq to get the italian wordssss
from wordfreq import top_n_list

In [3]:
# Get dataset
df_en = pd.read_csv('mainDataEN.csv')

df_it = pd.read_csv('haspeede2_dev_taskAB.tsv', sep='\t')

In [4]:
def is_hate_speech(value):
  if value['class'] == 2:
    return 0 #no hate speech
  else:
     return 1 #hate speech

df_en['hate_speech'] = df_en.apply(is_hate_speech, axis=1)

df_en = df_en.drop(['Unnamed: 0','count', 'offensive_language', 'neither', 'class'], axis=1)

df_en.head()

,hate_speech,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [5]:
df_it = df_it.drop(['id','stereotype'], axis=1)
df_it

,text,hs
0,"È terrorismo anche questo, per mettere in uno ...",0
1,@user @user infatti finché ci hanno guadagnato...,0
2,"Corriere: Tangenti, Mafia Capitale dimenticata...",0
3,"@user ad uno ad uno, perché quando i migranti ...",0
4,Il divertimento del giorno? Trovare i patrioti...,0
...,...,...
6832,Gli stati nazionali devono essere pronti a rin...,0
6833,Il ministro dell'interno della Germania #Horst...,0
6834,#Salvini: In Italia troppi si sono montati la ...,0
6835,@user @user Chi giubila in buona fede non ha c...,0


In [6]:
train_en, test_en = train_test_split(df_en, test_size=0.2)

X_train = train_en['tweet'].reset_index(drop=True)
Y_train = train_en['hate_speech'].reset_index(drop=True)

X_test = test_en['tweet'].reset_index(drop=True)
Y_test = test_en['hate_speech'].reset_index(drop=True)

X_it = df_it['text ']
Y_it = df_it['hs']

# THE BIBLE I FOLLOW

https://github.com/nlptown/nlp-notebooks/blob/master/Multilingual%20Embeddings%20-%203.%20Transfer%20Learning.ipynb

---

Start by downloading set of embeddings from the MUSE library, are fastText Wikipedia supervised word embeddings.


In [7]:
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec -O wiki.en.vec
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.it.vec -O wiki.it.vec

--2023-06-29 21:22:44--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 65.8.248.107, 65.8.248.22, 65.8.248.127, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|65.8.248.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628614720 (599M) [text/plain]
Saving to: ‘wiki.en.vec’

wiki.en.vec         100%[===================>] 599.49M  47.4MB/s    in 16s     

2023-06-29 21:23:01 (38.2 MB/s) - ‘wiki.en.vec’ saved [628614720/628614720]

--2023-06-29 21:23:01--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.it.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 65.8.248.107, 65.8.248.22, 65.8.248.127, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|65.8.248.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628718043 (600M) [text/plain]
Saving to: ‘wiki.it.vec’

wiki.it.vec         100%[===============

In [8]:
import numpy as np

def load_vectors(embedding_file_path):
    print("Loading vectors from", embedding_file_path)
    embeddings = []
    word2id = {}
    with open(embedding_file_path, 'r', encoding='utf-8') as f:
        next(f)
        for i, line in enumerate(f):
            word, emb = line.rstrip().split(' ', 1)
            emb = np.fromstring(emb, sep=' ')
            assert word not in word2id, 'word found twice'
            embeddings.append(emb)
            word2id[word] = len(word2id)

    embeddings = np.vstack(embeddings)
    return embeddings, word2id

# Produce the:
#   - embeddings_en → list of the embeddings of the words in wiki
#   - embedding_word2id_en → dictionary with each word and relative index in the embedding list
embeddings_en, embedding_word2id_en = load_vectors("wiki.en.vec")
embeddings_it, embedding_word2id_it = load_vectors("wiki.it.vec")


Loading vectors from wiki.en.vec
Loading vectors from wiki.it.vec


In [9]:
embeddings_en

array([[-0.0112864 , -0.00206967, -0.0515041 , ...,  0.0435525 ,
        -0.00774608,  0.0724234 ],
       [-0.0469256 , -0.00058526, -0.0750844 , ...,  0.0267626 ,
        -0.0513555 ,  0.0166127 ],
       [-0.0324474 , -0.0462027 , -0.00872643, ...,  0.0826505 ,
        -0.0649553 ,  0.0175795 ],
       ...,
       [-0.0608303 , -0.0172956 , -0.101635  , ...,  0.102298  ,
         0.0479459 ,  0.0387411 ],
       [-0.0867473 , -0.00469176, -0.104337  , ..., -0.00478122,
         0.130826  , -0.0207968 ],
       [-0.0119003 , -0.0123243 , -0.065963  , ...,  0.0184328 ,
        -0.0295059 , -0.0160871 ]])

In [10]:
embedding_word2id_en

{',': 0,
 '.': 1,
 'the': 2,
 '</s>': 3,
 'of': 4,
 '-': 5,
 'in': 6,
 'and': 7,
 "'": 8,
 ')': 9,
 '(': 10,
 'to': 11,
 'a': 12,
 'is': 13,
 'was': 14,
 'on': 15,
 's': 16,
 'for': 17,
 'as': 18,
 'by': 19,
 'that': 20,
 'it': 21,
 'with': 22,
 'from': 23,
 'at': 24,
 'he': 25,
 'this': 26,
 'be': 27,
 'i': 28,
 'an': 29,
 'utc': 30,
 'his': 31,
 'not': 32,
 '–': 33,
 'are': 34,
 'or': 35,
 'talk': 36,
 'which': 37,
 'also': 38,
 'has': 39,
 'were': 40,
 'but': 41,
 'have': 42,
 '#': 43,
 'one': 44,
 'rd': 45,
 'new': 46,
 'first': 47,
 'page': 48,
 'no': 49,
 'you': 50,
 'they': 51,
 'had': 52,
 'article': 53,
 't': 54,
 'who': 55,
 '?': 56,
 'all': 57,
 'their': 58,
 'there': 59,
 'been': 60,
 'made': 61,
 'its': 62,
 'people': 63,
 'may': 64,
 'after': 65,
 '%': 66,
 'other': 67,
 'should': 68,
 'two': 69,
 'score': 70,
 'her': 71,
 'can': 72,
 'would': 73,
 'more': 74,
 'if': 75,
 'she': 76,
 'about': 77,
 'when': 78,
 'time': 79,
 'team': 80,
 'american': 81,
 'such': 82,
 'th': 

In [11]:
# Deinfe some variables
VOCABULARY_SIZE = 10000
EMBEDDING_DIM = 300
SEQ_LENGTH = 300

INDEX_FROM = 3

In [12]:
X_train = ['<START> ' + x for x in X_train]
X_test= ['<START> ' + x for x in X_test]

In [13]:
# create the tokenizer
t = Tokenizer(oov_token="<unk>", num_words=VOCABULARY_SIZE+INDEX_FROM-1, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')

# fit the tokenizer on the documents
t.fit_on_texts(X_train)

t.word_index['<pad>'] = 0
t.index_word[0] = '<pad>'

# summarize what was learned
#print(t.word_counts)
#print(t.document_count)
print(t.word_index)
print(t.word_docs)

# integer encode documents
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

{'<unk>': 1, '<start>': 2, 'a': 3, 'bitch': 4, 'rt': 5, 'i': 6, 'the': 7, 'you': 8, 'to': 9, 'and': 10, 'my': 11, 'that': 12, 't': 13, '128514': 14, 'bitches': 15, 'in': 16, 'co': 17, 'is': 18, 'http': 19, 'like': 20, 'me': 21, 'on': 22, 'of': 23, 'hoes': 24, 'be': 25, 'pussy': 26, 'this': 27, 'for': 28, 'it': 29, 'hoe': 30, 'with': 31, '8220': 32, '8221': 33, "i'm": 34, 'ass': 35, 'all': 36, 'up': 37, 'if': 38, 'your': 39, 'but': 40, "don't": 41, 'just': 42, 'get': 43, 'fuck': 44, 'they': 45, 'so': 46, 'no': 47, 'when': 48, 'these': 49, 'shit': 50, 'u': 51, 'got': 52, 'nigga': 53, '8230': 54, 'not': 55, 'trash': 56, 'she': 57, 'are': 58, 'was': 59, 'her': 60, 'at': 61, 'lol': 62, 'out': 63, 'have': 64, 'about': 65, 'amp': 66, "ain't": 67, 'he': 68, 'what': 69, 'some': 70, 'know': 71, 'do': 72, 'niggas': 73, 'can': 74, 'we': 75, "it's": 76, 'how': 77, 'love': 78, 'one': 79, 'them': 80, 'who': 81, 'or': 82, 'as': 83, '128557': 84, 'fucking': 85, 'go': 86, 'yo': 87, 'now': 88, "can't": 8

In [14]:
X_train = tf.keras.utils.pad_sequences(X_train, maxlen=SEQ_LENGTH)
X_test = tf.keras.utils.pad_sequences(X_test, maxlen=SEQ_LENGTH)

In [15]:
len(X_train[76])

300

In [16]:
def create_embedding_matrix(target_word2id, embedding_word2id, embeddings, num_rows, num_columns):
    embedding_matrix = np.zeros((num_rows, num_columns))
    for word, i in target_word2id.items():
        if i >= num_rows:
            break
        if word in embedding_word2id:
            embedding_matrix[i] = embeddings[embedding_word2id[word]]
    return embedding_matrix

In [17]:
word2id_en = t.word_index

embedding_matrix_en = create_embedding_matrix(word2id_en, embedding_word2id_en, embeddings_en, VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM)

In [18]:
embedding_matrix_en

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.0388703 ,  0.00287599, -0.0371773 , ..., -0.0380095 ,
         0.0707945 , -0.0405729 ],
       [ 0.0265751 , -0.0130762 , -0.0582021 , ..., -0.00570843,
        -0.0434153 ,  0.0235883 ],
       [-0.113528  ,  0.0569072 ,  0.0824829 , ...,  0.0577218 ,
         0.0159746 , -0.0549382 ]])

In [19]:
# Model
model = Sequential()
# ------
model.add(keras.layers.Embedding(VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM, input_length=SEQ_LENGTH, weights=[embedding_matrix_en], trainable=False))
# ------
model.add(keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPooling1D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(250, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          3000600   
                                                                 
 conv1d (Conv1D)             (None, 300, 32)           28832     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 150, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 4800)              0         
                                                                 
 dense (Dense)               (None, 250)               1200250   
                                                                 
 dense_1 (Dense)             (None, 1)                 251       
                                                        

In [20]:
Y_train

0        1
1        1
2        1
3        1
4        1
        ..
19821    0
19822    1
19823    0
19824    1
19825    1
Name: hate_speech, Length: 19826, dtype: int64

In [21]:
NUM_EPOCHS=20
BATCH_SIZE=64

earlystop = EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
          epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[earlystop])

# Final evaluation of the model
scores = model.evaluate(X_test, Y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20
310/310 [==============================] - 53s 164ms/step - loss: 0.2210 - accuracy: 0.9033 - val_loss: 0.1531 - val_accuracy: 0.9348
Epoch 2/20
310/310 [==============================] - 53s 170ms/step - loss: 0.1349 - accuracy: 0.9448 - val_loss: 0.1375 - val_accuracy: 0.9409
Epoch 3/20
310/310 [==============================] - 49s 157ms/step - loss: 0.1138 - accuracy: 0.9547 - val_loss: 0.1423 - val_accuracy: 0.9381
Epoch 4/20
155/155 [==============================] - 3s 22ms/step - loss: 0.1527 - accuracy: 0.9381
Accuracy: 93.81%


FORSE SAREBBE BENE CONTROLLARE LE PERCENTUALI DI PRECISION/RECALL/ECC...

NON VORREI CHE METTESSE TIPO TUTTO HATE APPENA VEDE UNA PAOACIA :(

In [64]:
id = 15
tweet = np.array([X_test[id]])

print(test_en.iloc[id]['tweet'], " | ", test_en.iloc[id]['hate_speech'])
model.predict(tweet)

My goal for soccer this year is to get a yellow card every game  |  0
1/1 [==============================] - 0s 53ms/step


array([[0.0202361]], dtype=float32)

In [75]:
fake_tweet = "fuck you are a genious!!!!"
fake_tweet = t.texts_to_sequences([fake_tweet])
fake_tweet = tf.keras.utils.pad_sequences(fake_tweet, maxlen=SEQ_LENGTH)

model.predict(fake_tweet)

1/1 [==============================] - 0s 24ms/step


array([[0.9999184]], dtype=float32)

In [83]:
word2id_it = {word: idx+INDEX_FROM for idx, word in enumerate(top_n_list('it', VOCABULARY_SIZE))}
word2id_it["<pad>"] = 0
word2id_it["<start>"] = 2
word2id_it["<unk>"] = 1

embedding_matrix_it = create_embedding_matrix(word2id_it, embedding_word2id_it, embeddings_it, VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM)

In [84]:
model.layers[0].set_weights([embedding_matrix_it])

In [90]:
ita = t.texts_to_sequences(X_it)
ita = tf.keras.utils.pad_sequences(ita, maxlen=SEQ_LENGTH)

result_ita = model.predict(ita)

214/214 [==============================] - 5s 23ms/step


In [95]:
pepo = []
for res in result_ita:
  if res[0] >= 0.5:
    pepo.append(1)
  else:
    pepo.append(0)

In [118]:
right = 0
hihi = 0
for i in range(len(pepo)):
  if Y_it[i] == pepo[i]:
    right += 1
    if pepo[i] == 1:
      hihi = i

res = right/len(pepo)
print("Percentuale accuracy con ita: ", res)

Percentuale accuracy con ita:  0.5990931695187948


In [109]:
X_it[hihi]

'Basta andare a vedere come l\'islam hanno combinato il Libano;ex "Paradiso Terrestre" che in accoglienza Cittadinanza e Ricongiunzione Famigliare,in Pietà Cristiana concessero il diritto di voto agli immigratiBastò un lustro per vedersi buttare fuori da tutte le Istituzioni🔫😯 URL '

In [116]:
fake_tweet = ""
fake_tweet = t.texts_to_sequences([fake_tweet])
fake_tweet = tf.keras.utils.pad_sequences(fake_tweet, maxlen=SEQ_LENGTH)

model.predict(fake_tweet)

1/1 [==============================] - 0s 25ms/step


array([[0.07078993]], dtype=float32)

alora

non funziona molto bene anche se ho seguito per bene tutto quello che fa il tipo

secondo me è proprio per differenze linguistiche, teoricamente con i pesi dell'embedding in inglese ha capito come classificare i tweet riconoscendo la presenza di certe parole, e di conseguenza la loro posizione nello spazio dell'embedding

cambiando l'embedding lui si aspetta di trovare parole brutte in certe posizioni spaziali con determinate relazioni che probabilmente non si preservano nello spazio italiano

nonostante i risultati sia in inglese che in ita non siano poi così male, secondo me non performa comunque bene, ma credo la motivazione sia 50/50 dati e modello

i dati non sono un gran chè (come sono classificati) e con il nostro modello semplice semplice secondo me scegle le label in maniera conservativa, sparando 1 appena trova una minima parolaccia

in italiano non ho la più pallida idea di cosa reputi parolaccia, puoi scrivere le peggio cose e non lo labellllerà come offesa, sus......